# Crawling

- https://www.musinsa.com/app/goods/1108007


In [83]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
import pandas as pd

import warnings
warnings.filterwarnings("ignore") 

In [97]:
browser = webdriver.Chrome()
# browser.maximize_window()

url = 'https://www.musinsa.com/app/goods/1108007'
browser.get(url)

In [98]:
# 들어가자마자 뜨는 할인쿠폰 창 없애기
try:
    time.sleep(3)
    button = browser.find_element_by_class_name('ab-close-button')
    button.click()
except:
    button = None

In [ ]:
//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[3]

//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[7]
//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[8]

//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[5]

In [105]:
# 페이지 넘어가는 거 확인
review_list = []
for i in range(2):
    for i in range(3, 9 ,1):
        try:
            path = '//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[' + str(i) + ']'
            button = browser.find_element_by_xpath(path)
            button.click()
            print(path)
        except:
            button = None
            print('except')

//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[3]
//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[4]
//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[5]
//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[6]
//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[7]
//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[8]
//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[3]
//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[4]
//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[5]
//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[6]
//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[7]
//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[8]


In [108]:
# 11페이지까지 뽑아오는 시도
review_list = []

# 1페이지
html = browser.page_source
soup = BeautifulSoup(html,'lxml')
reviews = soup.find_all('div', "review-contents__text")

# 텍스트만 뽑음
for i in range(len(reviews)):
    review_list.append(str(reviews[i]).split('>')[1].split('<')[0].strip())

for i in range(2):
    for i in range(4, 9 ,1):
        try:
            path = '//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[' + str(i) + ']'
            button = browser.find_element_by_xpath(path)
            button.click()
            print(path)
        except:
            button = None
            print('except')
        
        html = browser.page_source
        soup = BeautifulSoup(html,'lxml')
        reviews = soup.find_all('div', "review-contents__text")
        
        # 텍스트만 뽑음
        for i in range(len(reviews)):
            review_list.append(str(reviews[i]).split('>')[1].split('<')[0].strip())

except
except
except
except
except
//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[4]
//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[5]
//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[6]
//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[7]
except


In [107]:
print(len(review_list))

print(pd.DataFrame(review_list).value_counts())

print(review_list)

120
말이 필요한가요 무탠다드 아이템은 늘 옳죠 가성비 갑                                                    2
좋아요 예쁘고요 사이즈도 잘 맞습니다                                                             2
오버핏을 선호하지 않아서 기본핏으로 구매했는데                                                        2
무난무난하규 결혼식셋업이나 차려입을 때 좋아요                                                        2
블레이저 처음 구매라면 정말 추천드립니다. 사이즈도 좋고 라인도 잘 빠지고 옷감도 고급진느낌이 납니다. 봄, 가을 입기에 좋은 두께감입니다    2
                                                                                ..
사이즈 한단계 업하시면 이뻐요 가성비 최고입니다                                                       1
사이즈때문에 걱정했는데 생각보다 잘 맞고 디자인도, 스타일도 다 좋네요.                                         1
살짝 요유로운게 맘에듭니다 사진보다 느낌은 더 작습니다                                                   1
생각보다 두껍고 무거운 느낌은 있지만 다른 블레이저 제품들에 비해 고급스러운 느낌은 확실히 드는 것 같습니다.                    1
핏도 좋고 옷도짱짱해서 계속 잘 입을것같습니다. 강추입니다.                                                1
Length: 89, dtype: int64
['큰 기대 안했는데 재질이며 기장이며 너무 마음에 듭니다!', '레귤러사이즈인데 두께가있어서 그런지 좀